In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A2_024, input_features_A2_024 = pre_models.Input_A2_024('weights/')
    
    outputs = ['Input_A2_024']

    preprocessings = ['skewness']

    models = [model_A2_024]

    input_features = [input_features_A2_024]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=4
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A2_024.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A2_022', 'Input_A2_023', 'Input_A3_024', 'Input_A3_022'],
      dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.01259847935796932
[0.006410197507050793, 0.012935488222901463, 0.0026527501289165083, 0.02718487499929699, 0.021028255419797102, 0.018732117544320925, 0.006170239543577961, 0.019571503768959223, 0.0041902039164528155, 0.007109162528419401]


test_r2_ave: -0.3357030632012184
[0.7739992174328645, 0.04228291990698507, 0.9666036727286333, -3.690654664202424, -0.9731526066852825, -1.1816671941401515, 0.8088801408429951, -1.740421116825941, 0.9177187610738936, 0.7193802378562444]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,4)

train_rmse: 0.003489222998819841
test_rmse: 0.003417772033754501
test_r2: 0.9344141176272135


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,9e-06)

test_rmse_ave: 0.003512979588446882
[0.003065687255470588, 0.003840134616679227, 0.0036056611113197906, 0.003302259540611516, 0.004082868698577859, 0.002855172819991618, 0.004039278998293442, 0.0031952380903410224, 0.003991219327767872, 0.0031522754254158874]


test_r2_ave: 0.9323147289206691
[0.9483080616152897, 0.9155957178090541, 0.9383013258659071, 0.9307848464242967, 0.9256150602238853, 0.9493149879750126, 0.9180951781091415, 0.9269575447153832, 0.925347957689839, 0.9448266087788821]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.005],0.01)

test_rmse_ave: 0.003848843260393845
[0.003285500750275861, 0.003975328275404609, 0.0037687809934287275, 0.003882227465814085, 0.004265407321435517, 0.003431537304458486, 0.004950556204171342, 0.00294131832076943, 0.004712132370766483, 0.0032756435974139104]


test_r2_ave: 0.9184153072310712
[0.9406295606888428, 0.9095481240550942, 0.9325925618745713, 0.9043377183005517, 0.9188151096193398, 0.9267863259329248, 0.8769703585900718, 0.9381053701648998, 0.8959443972358797, 0.9404235458485373]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,4,model)

train_rmse: 0.0037200815263964634
test_rmse: 0.0038325352436332483
test_r2: 0.9172259700493901


## XGB

In [8]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 160, 
              max_depth = 3, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.9,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.002610116594444569
[0.0020382978556536995, 0.0033939927210324184, 0.002543914919394047, 0.0027353326101223485, 0.001879888575991442, 0.002163334986991561, 0.0035740546294240664, 0.0021585374608674305, 0.00312541664734467, 0.002488395537624009]


test_r2_ave: 0.9610531636955469
[0.9771491470120371, 0.9340684332518828, 0.9692877818170784, 0.9525103393896454, 0.9842304824834702, 0.9709020562071885, 0.9358754781055965, 0.966665949112508, 0.9542231033133646, 0.9656188662626983]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 3,
            max_leaf_nodes = 8,
            min_samples_leaf =1,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.002786821328318193
[0.0020348403991184327, 0.0032690581491539206, 0.0026473960863721453, 0.0028936434983259924, 0.0024657709055810953, 0.0023529392579429874, 0.0039783915667526565, 0.0022150893975997755, 0.0035232157973223415, 0.00248786822501258]


test_r2_ave: 0.956100391487311
[0.9772266026440297, 0.9388330431088736, 0.9667383442257832, 0.9468542147571489, 0.9728693851396321, 0.9655779933476849, 0.920545807451898, 0.9648964183196689, 0.9418286698383145, 0.9656334360400753]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-06)
elastic_mod=ElasticNet(alpha=[0.005], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 160, 
              max_depth = 3, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.9,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 3,
            max_leaf_nodes = 8,
            min_samples_leaf =1,
            random_state = 42)  
# vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('SVR', svr_mod), ('Elastic', elastic_mod), 
#                             ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('Linear', lr),  
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.002681406657018221
[0.002398476302779021, 0.003134457807140367, 0.00276560258649376, 0.0024733293026977642, 0.003033732797652811, 0.0022565263519979946, 0.0034413633154755153, 0.001713292970075717, 0.003308851561778792, 0.0022884335740904673]


test_r2_ave: 0.9603435123811819
[0.9683598899792756, 0.9437663285255908, 0.9637017602271547, 0.9611722106036804, 0.9589314998916161, 0.9683411181720838, 0.940548499950217, 0.9789993769507618, 0.948692001316334, 0.9709224381951049]


In [10]:

stack_mod = StackingRegressor(regressors=[lasso_mod, random_mod,  vote_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.00213768385908183
[0.0019302008050881589, 0.002224880142713548, 0.0019457755234640838, 0.002639447802868007, 0.001331000991231841, 0.0016023181012437197, 0.0037172369412329207, 0.000944369456023605, 0.002898605440144966, 0.002143003386807453]


test_r2_ave: 0.9724502822303123
[0.9795085775900566, 0.971667504868285, 0.9820323412108587, 0.9557814053757548, 0.9920948348698345, 0.9840370839257593, 0.930634702040153, 0.9936195284269169, 0.9606260748484831, 0.9745007691470207]


In [11]:
ave(skew_data[title],y,0.7,stack_mod,0.15,vote_mod,0.15,xgb_mod)

test_rmse_ave: 0.0021671210086690257
[0.001926632476316881, 0.002358923520776988, 0.0020413675539699004, 0.002476695828243138, 0.0016147019514336396, 0.0017193470757543036, 0.003543077065435885, 0.001030838847179548, 0.0028592703888301067, 0.002100355378749866]


test_r2_ave: 0.9725583785114577
[0.9795842718328491, 0.9681507430595876, 0.9802235454692831, 0.9610664392414234, 0.988365735615498, 0.9816201542978887, 0.9369822395769624, 0.9923976043635759, 0.9616874588116304, 0.9755055928458775]


In [12]:
name='Input_A2_024'
test_pred=[]
seed=[0,2,4,5,7,1,9]
stack_w=0.7
vote_w=0.15
model_w=0.15
model=xgb_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)



final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))